# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


Submit "strawberry-backtest.tsv" in Gradescope.

In [20]:
# Part 1: Backtest Strawberry Prices  -> writes strawberry-backtest.tsv
# (Fixes dtype issue by forcing numeric arrays.)

import numpy as np
import pandas as pd

DATA_PATH = "/workspaces/dx704-project-02/strawberry-prices.tsv"  # Codespaces path

# --- Load & prep ---
df = pd.read_csv(DATA_PATH, sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

# Global time index (t) starting at 1
df["t"] = (
    (df["month"].dt.year - df["month"].iloc[0].year) * 12
    + (df["month"].dt.month - df["month"].iloc[0].month)
    + 1
)

# --- Design matrix: intercept + time trend + month dummies (Jan baseline) ---
def build_X(months: pd.Series, tvals: pd.Series) -> pd.DataFrame:
    m = months.dt.month.astype(int)
    dummies = pd.get_dummies(m, prefix="m", drop_first=True)  # m_2..m_12
    # Ensure all 11 dummy columns exist and in stable order
    for j in range(2, 13):
        col = f"m_{j}"
        if col not in dummies.columns:
            dummies[col] = 0
    dummies = dummies[[f"m_{j}" for j in range(2, 13)]]
    X = pd.concat(
        [
            pd.Series(1.0, index=tvals.index, name="intercept"),
            tvals.astype(float).rename("t"),
            dummies.astype(float),
        ],
        axis=1,
    )
    return X

def ols_predict(X_train: pd.DataFrame, y_train: pd.Series, X_new: pd.DataFrame) -> np.ndarray:
    # Force numeric arrays
    A = X_train.to_numpy(dtype=float)
    b = pd.to_numeric(y_train, errors="coerce").to_numpy(dtype=float)
    # Keep only rows where y is finite
    mask = np.isfinite(b)
    A = A[mask]
    b = b[mask]
    # Solve least squares
    beta, *_ = np.linalg.lstsq(A, b, rcond=None)
    return X_new.to_numpy(dtype=float) @ beta

# --- Split: train on 2020–2023, predict 2024 ---
train = df[df["month"].dt.year <= 2023].copy()
test  = df[df["month"].dt.year == 2024].copy()

X_tr = build_X(train["month"], train["t"])
y_tr = train["price"]
X_te = build_X(test["month"], test["t"])

pred_2024 = ols_predict(X_tr, y_tr, X_te)

# --- Save backtest file ---
backtest = pd.DataFrame({
    "month": test["month"].dt.strftime("%Y-%m"),
    "price": pred_2024
})
backtest.to_csv("strawberry-backtest.tsv", sep="\t", index=False)

print("Saved strawberry-backtest.tsv")
print(backtest)


Saved strawberry-backtest.tsv
      month     price
48  2024-01  4.783194
49  2024-02  4.511694
50  2024-03  4.109194
51  2024-04  4.207644
52  2024-05  3.946694
53  2024-06  3.695444
54  2024-07  3.614444
55  2024-08  3.910194
56  2024-09  4.002194
57  2024-10  4.346444
58  2024-11  4.818444
59  2024-12  5.209019


## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [21]:
# YOUR CHANGES HERE

# Part 2: Backtest Errors -> writes backtest-accuracy.tsv
# Residuals = actual (ground truth) - predicted
# Std is sample standard deviation (ddof=1).

import pandas as pd
import numpy as np

DATA_PATH = "/workspaces/dx704-project-02/strawberry-prices.tsv"   # Codespaces path to raw data
BACKTEST_PATH = "strawberry-backtest.tsv"                          # created in Part 1
OUTPUT_PATH = "backtest-accuracy.tsv"

# --- Load ground truth (filter 2024) ---
truth = pd.read_csv(DATA_PATH, sep="\t")
truth["month"] = pd.to_datetime(truth["month"])
truth_2024 = truth[truth["month"].dt.year == 2024].copy()
truth_2024["month_str"] = truth_2024["month"].dt.strftime("%Y-%m")
truth_2024 = truth_2024[["month_str", "price"]].rename(columns={"price": "actual"})

# --- Load predictions from Part 1 ---
pred = pd.read_csv(BACKTEST_PATH, sep="\t")
pred = pred.rename(columns={"month": "month_str", "price": "predicted"})

# --- Merge and compute residuals ---
df = truth_2024.merge(pred, on="month_str", how="inner").sort_values("month_str")
# Sanity check: should have 12 months
if len(df) != 12:
    print(f"Warning: expected 12 months for 2024, found {len(df)} after merge.")

df["residual"] = pd.to_numeric(df["actual"], errors="coerce") - pd.to_numeric(df["predicted"], errors="coerce")

# --- Mean and std of residuals ---
mean_resid = float(df["residual"].mean())
std_resid  = float(df["residual"].std(ddof=1))  # sample std

# --- Save to TSV ---
out = pd.DataFrame([{"mean": mean_resid, "std": std_resid}])
out.to_csv(OUTPUT_PATH, sep="\t", index=False)

print("Saved backtest-accuracy.tsv")
print(out)


Saved backtest-accuracy.tsv
     mean       std
0 -0.4253  0.274566


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [22]:
# YOUR CHANGES HERE

# Part 3: Forecast Strawberry Prices -> writes strawberry-forecast.tsv
# Methodology: OLS with linear trend + monthly seasonality (one-hot, drop_first)

import pandas as pd
import numpy as np

DATA_PATH = "/workspaces/dx704-project-02/strawberry-prices.tsv"  # Codespaces path
OUTPUT_PATH = "strawberry-forecast.tsv"

# --- Helpers (same as Part 1 methodology) ---
def build_X(month_series: pd.Series, t_series: pd.Series) -> pd.DataFrame:
    """
    Design matrix: intercept + linear trend t + monthly one-hots (drop_first to avoid collinearity).
    Ensures float dtype for numpy lstsq.
    """
    month_num = month_series.dt.month.astype(int)
    dummies = pd.get_dummies(month_num, prefix="m", drop_first=True).astype(float)  # m_2..m_12
    X = pd.DataFrame({
        "intercept": np.ones(len(t_series), dtype=float),
        "t": t_series.astype(float),
    })
    X = pd.concat([X, dummies], axis=1)
    # If any dummy columns are missing (edge case), add them with zeros to keep column set consistent
    for col in [f"m_{i}" for i in range(2, 13)]:
        if col not in X.columns:
            X[col] = 0.0
    # Order columns for stability
    X = X[["intercept", "t"] + [f"m_{i}" for i in range(2, 13)]]
    return X.astype(float)

def ols_predict(X_train: pd.DataFrame, y_train: pd.Series, X_new: pd.DataFrame) -> np.ndarray:
    beta, *_ = np.linalg.lstsq(X_train.values.astype(float), y_train.values.astype(float), rcond=None)
    return X_new.values.astype(float) @ beta

# --- Load all data (2020-2024) ---
df = pd.read_csv(DATA_PATH, sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)
df["t"] = np.arange(len(df))  # global time index across 2020-2024

# Train on 2020-2024 (all available)
train = df.copy()
X_tr = build_X(train["month"], train["t"])
y_tr = train["price"].astype(float)

# Build 2025 future grid
future_months = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
t_start = int(train["t"].iloc[-1]) + 1
t_future = pd.Series(np.arange(t_start, t_start + len(future_months)), index=future_months)

X_te = build_X(pd.Series(future_months), t_future.reset_index(drop=True))
pred_2025 = ols_predict(X_tr, y_tr, X_te)

# Save forecast
forecast = pd.DataFrame({
    "month": pd.Series(future_months).dt.strftime("%Y-%m"),
    "price": pred_2025
})
forecast.to_csv(OUTPUT_PATH, sep="\t", index=False)

print("Saved strawberry-forecast.tsv")
print(forecast.head())


Saved strawberry-forecast.tsv
     month     price
0  2025-01  4.750553
1  2025-02  4.375153
2  2025-03  3.948753
3  2025-04  3.994313
4  2025-05  3.717753


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [23]:
# YOUR CHANGES HERE

# Part 4: Buy Low, Freeze and Sell High -> writes timings.tsv
# Assumptions per spec:
# - Freeze cost: $0.20 per pint (fixed)
# - Storage cost: $0.10 per pint per month (charged for each month between buy and sell)
# - Sell price discount for frozen: 10% (i.e., realized sell price = 0.9 * forecast sell price)
# - Budget: $1,000,000 to cover ALL costs upfront (buy + freeze + storage); buy as many pints as possible.
# - Evaluate ALL month pairs buy<sell in 2025 forecast.

import pandas as pd
import numpy as np

FORECAST_PATH = "strawberry-forecast.tsv"  # expects columns: month (YYYY-MM), price
OUTPUT_PATH = "timings.tsv"

FREEZE_COST = 0.20
STORAGE_COST_PER_MONTH = 0.10
DISCOUNT = 0.10          # 10% discount
BUDGET = 1_000_000

# --- Load forecast for 2025 ---
f = pd.read_csv(FORECAST_PATH, sep="\t")
# Guard: ensure correct dtypes
f["month"] = pd.to_datetime(f["month"], format="%Y-%m")
f = f.sort_values("month").reset_index(drop=True)
f["price"] = f["price"].astype(float)

# Helper to compute integer month index (for differences)
def month_index(dt: pd.Timestamp) -> int:
    return dt.year * 12 + dt.month

rows = []
for i in range(len(f)):
    for j in range(i + 1, len(f)):
        buy_month = f.loc[i, "month"]
        sell_month = f.loc[j, "month"]
        buy_price = float(f.loc[i, "price"])
        sell_price = float(f.loc[j, "price"])

        # months between (assume whole months since monthly data)
        months_between = month_index(sell_month) - month_index(buy_month)
        if months_between <= 0:
            continue  # safety; we only consider buy < sell

        # Per-pint cash needed upfront (budget covers costs, not revenue)
        cost_per_pint_all_in = buy_price + FREEZE_COST + STORAGE_COST_PER_MONTH * months_between

        # Max pints purchasable within budget (integer)
        if cost_per_pint_all_in <= 0:
            # Degenerate edge case; skip
            continue
        pints = int(BUDGET // cost_per_pint_all_in)

        # Per-pint profit when sold
        realized_sell = (1.0 - DISCOUNT) * sell_price
        profit_per_pint = realized_sell - buy_price - FREEZE_COST - STORAGE_COST_PER_MONTH * months_between

        expected_profit = pints * profit_per_pint

        rows.append({
            "buy_month": buy_month.strftime("%Y-%m"),
            "sell_month": sell_month.strftime("%Y-%m"),
            "pints_purchased": pints,
            "expected_profit": expected_profit
        })

timings = pd.DataFrame(rows)
timings = timings.sort_values(["buy_month", "sell_month"]).reset_index(drop=True)

# Save results
timings.to_csv(OUTPUT_PATH, sep="\t", index=False)
print("Saved timings.tsv")
print(timings.head())


Saved timings.tsv
  buy_month sell_month  pints_purchased  expected_profit
0   2025-01    2025-02           197998   -220354.993670
1   2025-01    2025-03           194153   -309999.289813
2   2025-01    2025-04           190456   -315332.523590
3   2025-01    2025-05           186896   -374647.095924
4   2025-01    2025-06           183467   -427895.543292


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [24]:
# YOUR CHANGES HERE

# Part 5: Strategy Check -> writes check.tsv
# Reads:
#   - timings.tsv (from Part 4)
#   - strawberry-forecast.tsv (from Part 3)
#   - backtest-accuracy.tsv (from Part 2)
# Writes:
#   - check.tsv with columns: best_profit, one_std_profit

import pandas as pd
import numpy as np

TIMINGS_PATH = "timings.tsv"
FORECAST_PATH = "strawberry-forecast.tsv"
ACCURACY_PATH = "backtest-accuracy.tsv"
OUTPUT_PATH = "check.tsv"

# Constants (must match Part 4)
FREEZE_COST = 0.20
STORAGE_COST_PER_MONTH = 0.10
DISCOUNT = 0.10  # 10% discount on sell price for frozen goods

def month_index(dt: pd.Timestamp) -> int:
    return dt.year * 12 + dt.month

# --- Load inputs ---
timings = pd.read_csv(TIMINGS_PATH, sep="\t")
forecast = pd.read_csv(FORECAST_PATH, sep="\t")
acc = pd.read_csv(ACCURACY_PATH, sep="\t")

# Ensure dtypes
timings["buy_month"] = pd.to_datetime(timings["buy_month"], format="%Y-%m")
timings["sell_month"] = pd.to_datetime(timings["sell_month"], format="%Y-%m")
timings["expected_profit"] = timings["expected_profit"].astype(float)
timings["pints_purchased"] = timings["pints_purchased"].astype(int)

forecast["month"] = pd.to_datetime(forecast["month"], format="%Y-%m")
forecast["price"] = forecast["price"].astype(float)

std_resid = float(acc["std"].iloc[0])

# --- Find best scenario from Part 4 ---
best_idx = timings["expected_profit"].idxmax()
best_row = timings.loc[best_idx]
best_profit = float(best_row["expected_profit"])
buy_month = best_row["buy_month"]
sell_month = best_row["sell_month"]
pints = int(best_row["pints_purchased"])

# Lookup forecast prices for those months
buy_price = float(forecast.loc[forecast["month"] == buy_month, "price"].iloc[0])
sell_price = float(forecast.loc[forecast["month"] == sell_month, "price"].iloc[0])

# Months between
months_between = month_index(sell_month) - month_index(buy_month)

# --- Recompute profit if sell price is off by 1 std (downside) ---
sell_price_adj = max(0.0, sell_price - std_resid)  # guard against negative prices
realized_sell_adj = (1.0 - DISCOUNT) * sell_price_adj
profit_per_pint_adj = realized_sell_adj - buy_price - FREEZE_COST - STORAGE_COST_PER_MONTH * months_between
one_std_profit = pints * profit_per_pint_adj

# --- Save output ---
out = pd.DataFrame([{
    "best_profit": best_profit,
    "one_std_profit": one_std_profit
}])
out.to_csv(OUTPUT_PATH, sep="\t", index=False)
print("Saved check.tsv")
print(out)


Saved check.tsv
     best_profit  one_std_profit
0  101434.796572    41569.563742


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


In [25]:
# Part 6: Acknowledgments -> writes acknowledgments.txt

content = """Acknowledgments
==========================

People & Discussions
Ramsha Asad


Data Sources
- strawberry-prices.tsv (provided in course GitHub: dx704-project-02)

Generative AI Usage
- I used ChatGPT (GPT-5 Thinking) to help draft/refine code cells for Parts 1–5 (feature design, OLS setup, profit calc, file outputs).
- I reviewed and verified all outputs and made final decisions on methodology.

Notes
-none'.
"""

with open("acknowledgments.txt", "w", encoding="utf-8") as f:
    f.write(content)

print("Saved acknowledgments.txt")

Saved acknowledgments.txt


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.